In [1]:
from time import perf_counter
import pickle

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

from soynlp.tokenizer import LTokenizer
from PyKomoran import Komoran, DEFAULT_MODEL

from models.modified_sample.gpt2 import GPT2ModifiedSampleForCausalLM

In [2]:
# model_path = "lexiconium/kogpt-trinity"
# revision = "punct_wrapper-related_words-minevalloss"


# device = "cuda" if torch.cuda.is_available() else "cpu"
# tokenizer = AutoTokenizer.from_pretrained(model_path, revision=revision, use_auth_token=True)
# model = AutoModelForCausalLM.from_pretrained(model_path, revision=revision, use_auth_token=True).to(device)
# model.eval()

In [3]:
model_path = "/opt/ml/outputs/checkpoint-92"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2ModifiedSampleForCausalLM.from_pretrained(model_path).to(device)
model.eval()

GPT2ModifiedSampleForCausalLM(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplac

In [4]:
@torch.no_grad()
def genreate_from_input(
    input_text: str, max_length: int = 64, top_k: int = 30, top_p: float = 0.95, temperature: float = 1.0
):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Check generation time
    t = perf_counter()

    output = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
    )
    generated_text = tokenizer.decode(output[0])

    spent = perf_counter() - t

    return generated_text, spent


In [5]:
with open("/opt/ml/data/namuwiki_filtered_cohesion.pickle", "rb") as f:
    cohesions = pickle.load(f)
    l_cohesions = {word: score[0] for word, score in cohesions.items()}
    l_tokenizer = LTokenizer(l_cohesions)

In [6]:
komoran = Komoran(DEFAULT_MODEL["FULL"])

In [7]:
prompt = "스키를 타고 설산을 내려오는 남자"

# input_text = f"@{prompt}@<d>\n"

# words = l_tokenizer(prompt)
# input_text = f"@{', '.join(words)}@<d>\n"

nouns = komoran.nouns(prompt)
input_text = f"@{', '.join(nouns)}@<d>\n"

for top_p in [0.8]:
    for top_k in [16]:
        print(f"========\n{top_p=}\n{top_k=}\n========\n")
        for n in range(1, 6):
            generated_text, spent = genreate_from_input(input_text, max_length=64, top_k=top_k, top_p=top_p, temperature=1.0)
            print(f"[{n}]:\n{generated_text}\n")
            # print(f"\ntime spent: {spent:.2f} sec")


top_p=0.8
top_k=16

[1]:
@스키, 설산, 남자@<d> 
눈이 온다
설산 위로
남자가 스키를 타고
그 옆으로 눈이 쌓인다.
어딘지 모르게 슬퍼지는,
슬픈 눈.
여자의 얼굴엔 웃음이 가득하다.
나도 그 웃음 따라 웃었다.
나는 왜 이렇게 슬프고 슬픈지
왜 나는 이토록

[2]:
@스키, 설산, 남자@<d> 
남자 스키
내리는 눈 속에서,
하얀 눈을 맞으며.
이제 막 걸음마를 시작한 아이가 있는
그 보드라운 땅을 밟았다.
눈 덮인 산 속 어딘가에서-
아이가 미끄러져 내려왔다.
산길 위에서- 아이는 혼자였다.
한없이 울고 있는 아이

[3]:
@스키, 설산, 남자@<d> 
남자 스키어들이
이젠 남극에도 간다
설산도 아닌,
스키가 아니라.
여기는 남자가 아니란다.
그들은 그저 한 명일 뿐인데
누가 그들을 '남' 자로
불러주나?
우리에게 그들은 그냥 하나의 '존

[4]:
@스키, 설산, 남자@<d> 
스키는
겨울이면 더 이상 스키장이 아니다
눈을 뒤집어쓴 산과 들은
아직도 그 자리에 있다
사막은 아직도
지붕 없는 호텔로 남아있고
남자 한 명 없이,
여자들만 남은 채로
그저 눈 내리는 겨울에만

[5]:
@스키, 설산, 남자@<d> 
눈이 온다
설산 위를 걷는 스키어들이
발목을 담그고 간 물이
녹아들 즈음
얼음이 녹아내린다.
남자 또한 얼음장 위에 서서.
물기를 머금은 눈은
한순간에 남자의 두 눈을 가렸다.
겨울의 정취

